In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import random
import os
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt # 득점모델 변수 중요도
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb
from sklearn.model_selection import KFold

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [7]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y_01 Feature..
test_x = test_df.filter(regex='X')
test_y = test_df.filter(regex='Y')

In [16]:
train_x

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,...,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,...,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,...,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,...,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,...,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,...,1,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812
39603,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,...,1,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413
39604,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,...,1,1,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699
39605,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,0.0,...,1,1,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929


In [ ]:
[ 'X_02', 'X_48', 'X_47', 'X_23', 'X_11', 'X_10', 'X_04']

In [4]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(train_x.values, i) for i in range(train_x.shape[1])]
vif["features"] = train_x.columns
vif

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,3.338393,X_01
1,1.008929,X_02
2,2.185961,X_03
3,0.000000,X_04
4,1.766929,X_05
5,2.345513,X_06
6,1.024752,X_07
7,1.117746,X_08
8,1.177054,X_09
9,5.767093,X_10


In [7]:
remove_vif = vif[vif["VIF Factor"] >6]
remove_vif

,VIF Factor,features
12,23.233465,X_13
13,24.649917,X_14
14,11.183650,X_15
16,22.969227,X_17
17,14.625564,X_18
41,9.482675,X_42
42,8.465117,X_43
44,11.921528,X_45


In [8]:
rmlist = list(remove_vif.features.unique())
rmlist.extend(["X_04", "X_47", 'X_48', "X_10","X_11"])

In [9]:
rmlist

['X_13',
 'X_14',
 'X_15',
 'X_17',
 'X_18',
 'X_42',
 'X_43',
 'X_45',
 'X_04',
 'X_47',
 'X_48',
 'X_10',
 'X_11']

In [10]:
train_x.drop(rmlist,inplace=True,axis=1)
test_x.drop(rmlist,inplace=True,axis=1)

C:\Users\User\AppData\Local\Temp\ipykernel_16348\2183760222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x.drop(rmlist,inplace=True,axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_16348\2183760222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x.drop(rmlist,inplace=True,axis=1)


In [12]:
# from sklearn.preprocessing import StandardScaler
# train_x = StandardScaler().fit_transform(train_x)
# train_x = pd.DataFrame(data = train_x)

In [12]:
train_x

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0.500000,0.0,0.336391,0.085137,0.520025,0.102258,0.010183,0.346935,0.318182,0.400000,...,0.410256,0.500000,0.057221,0.818378,0.657283,0.829713,0.861965,0.678313,0.772826,0.774036
1,0.464276,1.0,0.266055,0.122655,0.440003,0.097449,0.009694,0.326766,0.500000,0.571429,...,0.512821,0.500000,0.063671,0.794895,0.660879,0.843594,0.780464,0.693197,0.817679,0.772908
2,0.571414,0.0,0.232416,0.994949,0.440003,0.097983,0.028655,0.391092,0.409091,0.371429,...,0.487179,0.916667,0.068392,0.788280,0.632348,0.822906,0.820963,0.687465,0.797791,0.753194
3,0.535689,0.0,0.339450,0.142136,0.600008,0.098718,0.032674,0.363021,0.272727,0.371429,...,0.410256,1.000000,0.104882,0.797750,0.645002,0.775716,0.846305,0.711607,0.846910,0.847593
4,0.464276,0.0,0.217125,0.149351,0.360020,0.103794,0.027638,0.339851,0.363636,0.428571,...,0.435897,1.000000,0.069314,0.858752,0.664588,0.743346,0.865374,0.678588,0.790558,0.760922


In [11]:
from sklearn.preprocessing import MinMaxScaler
train_x = MinMaxScaler().fit_transform(train_x)
train_x = pd.DataFrame(data = train_x)
test_x = MinMaxScaler().fit_transform(test_x)
test_x = pd.DataFrame(data = test_x)


In [17]:
xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=15, learning_rate=0.1, n_jobs=-1)
kfold = KFold(n_splits=5, shuffle=True)
kfold_regr = MultiOutputRegressor(xgb_model)

In [18]:
loss_fold = []

for train, test in kfold.split(train_x):
    XGB = MultiOutputRegressor(xgb_model).fit(train_x.loc[train], train_y.loc[train])
    train_predict = XGB.predict(train_x.loc[test])
    scores = math.sqrt(mean_squared_error(train_predict, train_y.loc[test]))
    loss_fold.append(scores)

KeyboardInterrupt: 

In [19]:
XGB = MultiOutputRegressor(xgb_model).fit(train_x, train_y)

In [20]:
loss_fold

[1.2014713185464654, 1.2130061647956343, 1.2201462936653105, 1.213047933572596]

In [21]:
test_predict = XGB.predict(test_x)


In [22]:
submit = pd.read_csv('./sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = test_predict[:,idx-1]
print('Done.')

Done.


In [23]:
submit.to_csv('xgb_vif_minmax.csv', index=False)

In [130]:
from sklearn.model_selection import GridSearchCV
 
param_grid = [
        {'max_depth' : [5,6,7,8],  'learning_rate':[0.01, 0.1], 'random_state':[42], "n_jobs":[-1]}
        
    ]
forest_reg = MultiOutputRegressor(xgb.XGBRegressor())
 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                           n_jobs = -1)
 
grid_search.fit(train_x, train_y)

ValueError: Invalid parameter 'learning_rate' for estimator MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_to_onehot=None,
                          

In [16]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler



gsc = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4],},
            cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result = MultiOutputRegressor(gsc).fit(train_x, train_y)
self.best_params = grid_result.estimators_[0].best_params_


KeyboardInterrupt: 